In [1]:
import sys
sys.path.append('..')
import unittest
import os
import numpy as np
import pickle
from Compound_n import Compound
from ModificationSiteLocator import ModificationSiteLocator
from SiteLocator import SiteLocator
import utils

# library = "GNPS-LIBRARY"
library = "GNPS-MSMLS"
# library = "GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE"

# load data_dict_filtered
with open(os.path.join("../data/libraries",library,"data_dict_filtered.pkl"), "rb") as f:
    data_dict_filtered = pickle.load(f)

# load matches
with open(os.path.join("../data/libraries",library,"matches.pkl"), "rb") as f:
    matches = pickle.load(f)

# load cachedStructures_filtered
with open(os.path.join("../data/libraries",library,"cachedStructures.pkl"), "rb") as f:
    cachedStructures_filtered = pickle.load(f)



args = {
    'filter_peaks_method':"top_k",
      'filter_peaks_variable':60,
        'mz_tolerance':0.05,
        'mz_tolerance':0.05
}
peak_presence_only = False
consider_intensity = False
combine = False

In [11]:
m0 = "CCMSLIB00005464384"
m1 = "CCMSLIB00005464404"

In [12]:
main_compound = Compound(data_dict_filtered[m0], cachedStructures_filtered[m0], args)
modified_compound = Compound(data_dict_filtered[m1], cachedStructures_filtered[m1], args)
site_locator = ModificationSiteLocator(main_compound, modified_compound, args)
new_scores = site_locator.generate_probabilities(shifted_only=(not combine), PPO=peak_presence_only, CI=consider_intensity)
modification_site = utils.calculateModificationSites(modified_compound.structure, main_compound.structure, False)[0]

molMol = cachedStructures_filtered[m0]
modifMol = cachedStructures_filtered[m1]
molSmiles = data_dict_filtered[m0]['Smiles']
modifSmiles = data_dict_filtered[m1]['Smiles']
site = SiteLocator(data_dict_filtered[m0], data_dict_filtered[m1], molSmiles, args)

scores_unshifted, scores_shifted = site.calculate_score(peak_presence_only, consider_intensity)
old_scores = site.distance_score(scores_unshifted, scores_shifted, combine)
old_scores = old_scores - np.min(old_scores)
if np.sum(old_scores) != 0:
    old_scores = old_scores / np.sum(old_scores)
else:
    old_scores = np.zeros(len(old_scores))

here?
here?


In [4]:
print(len(main_compound.peaks))
print(len(site.molPeaks))

60
60


In [5]:
print(site.shifted)
print( site_locator.shifted)

[(24, 18), (50, 51), (46, 44), (36, 25), (16, 12), (42, 33), (28, 21), (44, 41), (23, 17), (56, 55), (26, 19), (51, 50), (32, 22), (43, 34), (35, 23), (25, 20), (37, 28)]
[(24, 18), (50, 51), (46, 44), (36, 25), (16, 12), (42, 33), (28, 21), (44, 41), (23, 17), (56, 55), (26, 19), (51, 50), (32, 22), (43, 34), (35, 23), (25, 20), (37, 28)]


In [6]:
scores_unshifted

array([ 9., 14., 14., 14.,  9.,  7.,  7., 12.])

In [7]:
scores_shifted

array([22., 54., 50., 59., 46., 37., 36., 44.])

In [8]:
s_peakids = [_[0] for _ in site_locator.shifted]
positive_contributions = site_locator.calculate_contributions(s_peakids, peak_presence_only, consider_intensity)
positive_contributions

[22, 54, 50, 59, 46, 37, 36, 44]

In [10]:
np.allclose(new_scores, old_scores)

True